<a href="https://colab.research.google.com/github/atadria/AGH_data_science/blob/master/PytorchWine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pytorch project 

##Using chemical analysis determine the origin of wines

https://archive.ics.uci.edu/ml/datasets/wine


###Imports

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [0]:
import pandas as pd
import numpy as np
import torchvision
import torch

##Read data from csv
https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data

In [42]:
data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
wine_data = pd.read_csv(data_url, header=None)
train_data = wine_data.sample(frac=0.8)
test_data = wine_data.drop(train_data.index)
train_classes = train_data[0]
train_data = train_data.drop(0, axis = 1)
test_classes = test_data[0]
test_data = test_data.drop(0, axis = 1)
test_data.head(10)

,1,2,3,4,5,6,7,8,9,10,11,12,13
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
7,14.06,2.15,2.61,17.6,121,2.60,2.51,0.31,1.25,5.05,1.06,3.58,1295
9,13.86,1.35,2.27,16.0,98,2.98,3.15,0.22,1.85,7.22,1.01,3.55,1045
21,12.93,3.80,2.65,18.6,102,2.41,2.41,0.25,1.98,4.50,1.03,3.52,770
28,13.87,1.90,2.80,19.4,107,2.95,2.97,0.37,1.76,4.50,1.25,3.40,915
30,13.73,1.50,2.70,22.5,101,3.00,3.25,0.29,2.38,5.70,1.19,2.71,1285
33,13.76,1.53,2.70,19.5,132,2.95,2.74,0.50,1.35,5.40,1.25,3.00,1235
37,13.05,1.65,2.55,18.0,98,2.45,2.43,0.29,1.44,4.25,1.12,2.51,1105
45,14.21,4.04,2.44,18.9,111,2.85,2.65,0.30,1.25,5.24,0.87,3.33,1080
49,13.94,1.73,2.27,17.4,108,2.88,3.54,0.32,2.08,8.90,1.12,3.10,1260


###Normalize data

In [86]:
train_data = train_data.apply(lambda x: 10*(x - x.min()) / (x.max() - x.min()))
test_data = test_data.apply(lambda x: 10*(x - x.min()) / (x.max() - x.min()))
test_data.head(10)

,1,2,3,4,5,6,7,8,9,10,11,12,13
2,5.372549,4.068768,8.522727,2.08,2.738095,7.021277,9.016393,2.500000,8.356643,3.345455,5.925926,8.209607,8.905473
7,8.901961,3.467049,7.840909,1.28,5.119048,6.170213,6.622951,2.727273,2.902098,2.772727,6.296296,10.000000,10.000000
9,8.117647,1.174785,3.977273,0.00,2.380952,7.787234,8.721311,0.681818,5.000000,4.745455,5.679012,9.868996,7.512438
21,4.470588,8.194842,8.295455,2.08,2.857143,5.361702,6.295082,1.363636,5.454545,2.272727,5.925926,9.737991,4.776119
28,8.156863,2.750716,10.000000,2.72,3.452381,7.659574,8.131148,4.090909,4.685315,2.272727,8.641975,9.213974,6.218905
30,7.607843,1.604585,8.863636,5.20,2.738095,7.872340,9.049180,2.272727,6.853147,3.363636,7.901235,6.200873,9.900498
33,7.725490,1.690544,8.863636,2.80,6.428571,7.659574,7.377049,7.045455,3.251748,3.090909,8.641975,7.467249,9.402985
37,4.941176,2.034384,7.159091,1.60,2.380952,5.531915,6.360656,2.272727,3.566434,2.045455,7.037037,5.327511,8.109453
45,9.490196,8.882521,5.909091,2.32,3.928571,7.234043,7.081967,2.500000,2.902098,2.945455,3.950617,8.908297,7.860697
49,8.431373,2.263610,3.977273,1.12,3.571429,7.361702,10.000000,2.954545,5.804196,6.272727,7.037037,7.903930,9.651741


##Load data

In [81]:
train_set = torch.utils.data.TensorDataset(torch.tensor(train_data.values).float(), torch.tensor(train_classes.values - 1))
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)

test_set = torch.utils.data.TensorDataset(torch.tensor(test_data.values).float(), torch.tensor(test_classes.values - 1))
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=128)

torch.tensor(train_classes.values -1)

tensor([2, 0, 1, 2, 1, 1, 2, 0, 0, 1, 2, 0, 0, 2, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
        1, 1, 1, 2, 2, 0, 2, 1, 2, 1, 0, 2, 0, 0, 1, 2, 1, 0, 0, 1, 1, 2, 1, 1,
        0, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 0, 0, 0, 1,
        1, 0, 2, 0, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        1, 1, 0, 0, 1, 2, 1, 2, 0, 0, 0, 0, 0, 1, 1, 2, 1, 2, 0, 0, 0, 0, 2, 0,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 2, 1, 0, 2, 0, 2, 2, 2, 0, 0, 1, 0])

##Define model

In [92]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # linear layer
        self.fc1 = nn.Linear(13, 15)
        # linear layer
        self.fc2 = nn.Linear(15, 10)
        # linear layer
        self.fc3 = nn.Linear(10, 3)
        # dropout layer
        self.dropout = nn.Dropout(0.1)
        

    def forward(self, x):
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        #x = F.softmax(x, dim=1)
        return x

model = Net()
print(model)

if torch.cuda.is_available():
  model.cuda()

Net(
  (fc1): Linear(in_features=13, out_features=15, bias=True)
  (fc2): Linear(in_features=15, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
  (dropout): Dropout(p=0.1)
)


In [0]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 100 ,gamma=0.8, last_epoch=-1)

In [96]:
# number of epochs to train the model
n_epochs = 500
valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for batch_idx, (data, target) in enumerate(train_dataloader):
        # move tensors to GPU if CUDA is available
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        scheduler.step()
        # update training loss
        train_loss += loss.item()*data.size(0)    
    ######################    
    # validate the model #
    ######################
    model.eval()
    accuracy = 0
    with torch.no_grad():
      for batch_idx, (data, target) in enumerate(test_dataloader):
          # move tensors to GPU if CUDA is available
          if torch.cuda.is_available():
              data, target = data.cuda(), target.cuda()
          # forward pass: compute predicted outputs by passing inputs to the model
          output = model(data)
          # calculate the batch loss
          loss = criterion(output, target)
          # update average validation loss 
          valid_loss += loss.item()*data.size(0)
          #calculate accuracy 
          ps = torch.exp(model.forward(data))
          top_p, top_class = ps.topk(1, dim=1)
          equals = top_class == target.view(*top_class.shape)
          accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
    
    # calculate average losses
    train_loss = train_loss/len(train_dataloader.dataset)
    valid_loss = valid_loss/len(test_dataloader.dataset)
    accuracy = accuracy/len(test_dataloader) 
   
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tAccuracy: {:.6f}'.format(
        epoch, train_loss, valid_loss, accuracy))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_augmented.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 0.097174 	Validation Loss: 0.200832 	Accuracy: 0.944444
Validation loss decreased (inf --> 0.200832).  Saving model ...
Epoch: 2 	Training Loss: 0.097767 	Validation Loss: 0.221213 	Accuracy: 0.944444
Epoch: 3 	Training Loss: 0.131151 	Validation Loss: 0.224969 	Accuracy: 0.944444
Epoch: 4 	Training Loss: 0.117419 	Validation Loss: 0.227009 	Accuracy: 0.944444
Epoch: 5 	Training Loss: 0.092567 	Validation Loss: 0.217462 	Accuracy: 0.944444
Epoch: 6 	Training Loss: 0.116927 	Validation Loss: 0.228115 	Accuracy: 0.916667
Epoch: 7 	Training Loss: 0.097832 	Validation Loss: 0.207601 	Accuracy: 0.916667
Epoch: 8 	Training Loss: 0.162228 	Validation Loss: 0.227672 	Accuracy: 0.916667
Epoch: 9 	Training Loss: 0.081671 	Validation Loss: 0.228723 	Accuracy: 0.944444
Epoch: 10 	Training Loss: 0.124074 	Validation Loss: 0.194301 	Accuracy: 0.944444
Validation loss decreased (0.200832 --> 0.194301).  Saving model ...
Epoch: 11 	Training Loss: 0.123423 	Validation Loss: 0.2